In [ ]:
import pandas as pd

calendar = pd.read_csv('../data/calendar.csv')
inventory = pd.read_csv('../data/inventory.csv')
test = pd.read_csv('../data/sales_test.csv')
train = pd.read_csv('../data/sales_train.csv')
solution = pd.read_csv('../data/solution.csv')
weights = pd.read_csv('../data/test_weights.csv')

In [9]:
print("CALENDAR")
print(calendar.info())
print("--------------------------------")

print("INVENTORY")
print(inventory.info())
print("--------------------------------")

print("TEST")
print(test.info())
print("--------------------------------")

print("TRAIN")
print(train.info())
print("--------------------------------")

print("SOLUTION")
print(solution.info())
print("--------------------------------")

print("WEIGHTS")
print(weights.info())
print("--------------------------------")




CALENDAR
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    23016 non-null  object
 1   holiday_name            930 non-null    object
 2   holiday                 23016 non-null  int64 
 3   shops_closed            23016 non-null  int64 
 4   winter_school_holidays  23016 non-null  int64 
 5   school_holidays         23016 non-null  int64 
 6   warehouse               23016 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.2+ MB
None
--------------------------------
INVENTORY
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5432 entries, 0 to 5431
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   unique_id            5432 non-null   int64 
 1   product_unique_id    5432 non-null   int64 
 2   name      